In [8]:
import os
import sys
import random
import re
import omero
import requests
import imgkit
import random

import numpy as np
import tensorflow as tf

import PIL.Image
from io import BytesIO
from time import sleep

from pandas import Series
from pandas import DataFrame
from pandas import read_csv
from pandas import merge,concat
from pandas import read_hdf,HDFStore

from idr import connection, images_by_phenotype, attributes_by_attributes, simple_colocalisation

import pandas as pd
pd.set_option('display.max_columns', 500)

ModuleNotFoundError: No module named 'omero'

Download file images from IDR using their package which is built on OMERO

In [ ]:
# Create connection to IDR
conn = connection('idr.openmicroscopy.org', 'public', 'public')

In [2]:
# Create a dataframe of metadata annotation for the images that are in the screen of interest
def getBulkAnnotationAsDf(screenID, conn):
    ofId = None
    sc = conn.getObject('Screen', screenID)
    for ann in sc.listAnnotations():
        if isinstance(ann, omero.gateway.FileAnnotationWrapper):
            if (ann.getFile().getName() == 'bulk_annotations'):
                if (ann.getFile().getSize()> 1476250900): #about 140Mb?
                    print("that's a big file...")
                    return None
                ofId = ann.getFile().getId()
                break

    if ofId is None:
        return None

    original_file = omero.model.OriginalFileI(ofId, False)

    table = conn.c.sf.sharedResources().openTable(original_file)
    count = 0
    try:
        rowCount = table.getNumberOfRows()

        column_names = []
        pattern = re.compile("Phenotype \d+$")
        for col in table.getHeaders():
            column_names.append(col.name)
            if pattern.match(col.name) is not None:
                count = count + 1

        black_list = []
        column_indices = []
        for column_name in column_names:
            if column_name in black_list:
                continue
            column_indices.append(column_names.index(column_name))

        table_data = table.slice(column_indices, None)
    finally:
        table.close()

    data = []
    for index in range(rowCount):
        row_values = [column.values[index] for column in table_data.columns]
        data.append(row_values)

    dfAnn = DataFrame(data)
    dfAnn.columns = column_names
    return dfAnn, count

In [ ]:
df_meta, phenotype_count = getBulkAnnotationAsDf(2651, conn)

In [ ]:
df_meta

Now identify the images to be downloaded

In [ ]:
df_meta.to_pickle('df_meta.pickle')

In [40]:
df_meta = pd.read_pickle('df_meta.pickle')

FileNotFoundError: [Errno 2] No such file or directory: 'df_meta.pickle'

In [ ]:
# Get a list of all of the plates included in the screen
screen = conn.getObject("Screen", 2651)
plateIds = []
for plate in screen.listChildren():
    plateId = plate.getId()
    plateIds.append(plateId)

In [ ]:
# Get a list of all images on the plates
imageIds = []
for plateId in plateIds:
    plate = conn.getObject("Plate", plateId)
    # print("\nNumber of fields:", plate.getNumberOfFields())
    # print("\nGrid size:", plate.getGridSize())
    # print("\nWells in Plate:", plate.getName())
    for well in plate.listChildren():
        index = well.countWellSample()
    #     print("  Well: ", well.row, well.column, " Fields:", index)
        for index in range(0, index):
    #         print("    Image: ", \
    #             well.getImage(index).getName(),\
    #             well.getImage(index).getId())
            imageIds.append(well.getImage(index).getId())

In [ ]:
len(imageIds)
with open('imageIds.txt', 'w') as f:
    for item in imageIds:
        f.write("%s\n" % item)

In [ ]:
def url2array(image_name):
    url_ = 'https://idr.openmicroscopy.org/webgateway/render_image/' + str(image_name) + '/0/0/'

    response = requests.get(url_)
    img = PIL.Image.open(BytesIO(response.content))


    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = mobilenet_v2.preprocess_input(x)
    
    
    final_image_location = 'metis-deeplearning/processed_imgs/' + str(image_name) 
    np.save(final_image_location, x)
#     file_ = open(final_image_location, 'wb')
#     file_.write(x)
#     file_.close()  
    return x

In [ ]:
def get_png_from_url(url_, directory_, image_name):
    request_info = requests.get(url_)
    final_image_location = directory_ + str(image_name) + '.png'
    file_ = open(final_image_location, 'wb')
    file_.write(request_info.content)
    file_.close()
    
url_
directory_ = 'raw_images/'
image_name = image_
get_png_from_url(url_, directory_, image_name)

In [ ]:
i = 0
for image_ in imageIds[25500:100000]:
    i += 1
    url_ = 'https://idr.openmicroscopy.org/webgateway/render_image/' + str(image_) + '/0/0/'
    directory_ = 'raw_images/'
    image_name = str(image_)
    get_png_from_url(url_, directory_, image_name)
    if i % 100 == 0:
        sleep(random.randint(10,30))
        print(i)

Download images as jpegs and name by ID

In [64]:
i = 0
for image_ in imageIds[10:10000]:
    i += 1
    url2array(image_)
    if i % 100 == 0:
        sleep(random.randint(10,30))
    else:
        print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279


KeyboardInterrupt: 

In [63]:
np.load('metis-deeplearning/processed_imgs/11340519.npy')

array([[[[-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.],
         ...,
         [-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.]],

        [[-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.],
         ...,
         [-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.]],

        [[-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.],
         ...,
         [-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.]],

        ...,

        [[-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.],
         ...,
         [-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.]],

        [[-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.],
         ...,
         [-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.]],

        [[-1., -1., -1.],
         [-1., -1., -1.],
         [-1., -1., -1.],
         ...,
         [-1., -1., -1.],
        

In [7]:
# Load in image ids from the saved file
imageIds = []
from ast import literal_eval
infile = open('imageids.txt', 'r')
for line in infile:
    imageIds. append(line.replace('\n', ''))
infile.close()
print(imageIds[:5])

['11340528', '11340529', '11340530', '11340525', '11340526']


In [31]:
INDEX_PAGE = "https://idr.openmicroscopy.org/webclient/?experimenter=-1"

# create http session
with requests.Session() as session:
    request = requests.Request('GET', INDEX_PAGE)
    prepped = session.prepare_request(request)
    response = session.send(prepped)
    if response.status_code != 200:
        response.raise_for_status()

image_drug_dict = {}
i = 0
for image in imageIds[0:26000]:
    i += 1
    if i % 1000 == 0:
        print(i)
    image_id = 1957452

    MAP_URL = "https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&{type}={image_id}"

    qs = {'type': 'image', 'image_id': image}
    url = MAP_URL.format(**qs)
    for a in session.get(url).json()['annotations']:
        namespace = a['ns']
        for v in a['values']:
            key = v[0]
            if key == 'Compound Name':
                value = v[1]
#                 print(key, value)
                image_drug_dict[image] = v[1]

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000


In [29]:
image_drug_dict

{'11421776': 'AG-1024',
 '11421777': 'AG-1024',
 '11421772': 'CK 869',
 '11421773': 'CK 869',
 '11421774': 'CK 869',
 '11421769': 'PKI 166 hydrochloride',
 '11421770': 'PKI 166 hydrochloride',
 '11421771': 'PKI 166 hydrochloride',
 '11421766': 'LRRK2-IN-1',
 '11421767': 'LRRK2-IN-1',
 '11421768': 'LRRK2-IN-1',
 '11421763': 'methazolamide',
 '11421764': 'methazolamide',
 '11421765': 'methazolamide',
 '11421760': 'DMSO',
 '11421761': 'DMSO',
 '11421762': 'DMSO',
 '11421757': 'Kaempferol',
 '11421758': 'Kaempferol',
 '11421759': 'Kaempferol',
 '11421754': 'K 252a',
 '11421755': 'K 252a',
 '11421756': 'K 252a',
 '11421751': 'Cytochalasin B',
 '11421752': 'Cytochalasin B',
 '11421753': 'Cytochalasin B',
 '11421748': 'NSC 109555 ditosylate',
 '11421749': 'NSC 109555 ditosylate',
 '11421750': 'NSC 109555 ditosylate',
 '11421745': 'EC 144',
 '11421746': 'EC 144',
 '11421747': 'EC 144',
 '11421742': 'K 252a',
 '11421743': 'K 252a',
 '11421744': 'K 252a',
 '11421739': 'Leucomisine',
 '11421740':

In [32]:
image_drug_df = pd.DataFrame(image_drug_dict.items(), columns = ['Image', 'Drug'])

In [33]:
image_drug_df.shape

(26000, 2)

In [79]:
df_MoA_map = df_meta[['Compound MoA', 'Compound Name']].drop_duplicates()
df_MoA_map.to_csv('MoA_map.csv')

In [34]:
MoA_map_complete = pd.read_csv('MoA_map_complete.csv')

In [35]:
df_image_MoA = pd.merge(image_drug_df, MoA_map_complete[['Compound MoA', 'Compound Name']], left_on = 'Drug', right_on = 'Compound Name',  how = 'left')

In [36]:
df_image_MoA.to_csv('df_image_MoA.csv')

In [37]:
df_image_MoA['Compound MoA'].value_counts()

Kinase inhibitor                                  2930
Chromatin structure                               1782
GPCR agonist                                      1641
GPCR antagonist                                   1437
Channel blocker                                   1161
Signaling inhibitor                                996
DNA/RNA inhibitor                                  996
Hormone release inhibitor                          729
Cell cycle inhibitor                               600
Metabolic inhibitor                                495
Growth factor inhibitor                            486
Transcription factor agonist                       390
Tyrosine kinase inhibitor                          336
DNA repair inhibitor                               264
Chaperone inhibitor                                258
Cytoskeleton inhibitor                             243
Motor inhibitor                                    237
Kinase activator                                   204
Microbicid

In [11]:
# First focus on just one cell type
df_A549 = df_meta[df_meta['Characteristics [Cell Line]'] == 'A549']
df_A549.Plate.nunique()

400